# The Night That Taggart and I Reversed Nimble.exe

To finish this presentation, let me tell you a story.

January 5th, 2021. John Hammond tweets the following:

![johntweet](../assets/NimbleAVExcursion/johntweet.png)

Upon closer inspection, John is attempting to download Nim version 1.6.2 (the current stable build as of writing this)


---
## [Previous: Nim's Offensive Applications](OffensiveNim.ipynb) ||--0--|| [Next: Conclusion](Conclusion.ipynb)